# 7. 패딩(Padding)
텍스트 데이터에 대해서 정수 인코딩을 수행하고나면, 이제 각 텍스트는 정수 시퀀스로 변환된 상태입니다. 정수 시퀀스로 변환된 각 문장(또는 문서)는 서로 길이가 다를 수 있습니다. 그런데 기계는 길이가 전부 동일한 문서들에 대해서는 하나의 행렬로 보고, 한꺼번에 묶어서 처리할 수 있습니다. 다시 말해 병렬 연산을 위해서 각 문서의 길이를 동일하게 맞춰주는 작업이 필요할 때가 있습니다.

## 1) Numpy로 패딩하기

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

sentences = [['barber', 'person'], ['barber', 'good', 'person'], 
             ['barber', 'huge', 'person'], ['knew', 'secret'], 
             ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], 
             ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], 
             ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], 
             ['barber', 'went', 'huge', 'mountain']]

# 정수 인코딩 작업
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences) # fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성한다.

# 텍스트 시퀀스의 모든 단어들을 각 정수로 맵핑 후 출력
encoded = tokenizer.texts_to_sequences(sentences)
print(encoded) # 모든 단어가 고유한 정수로 변환

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


In [2]:
# 모두 동일한 길이로 맞춰주기 위해서 이 중에서 가장 길이가 긴 문장의 길이를 계산

max_len = max(len(item) for item in encoded)
print(max_len)

7


가장 길이가 긴 문장의 길이는 7입니다. 이제 모든 문장의 길이를 7로 맞춰주겠습니다. 이때 가상의 단어 'PAD'를 사용합니다. 'PAD'라는 단어가 있다고 가정하고, 이 단어는 0번 단어라고 정의해보겠습니다. 이제 길이가 7보다 짧은 문장에는 숫자 0을 채워서 전부 길이 7로 맞춰주겠습니다.

In [13]:
for item in encoded: # 각 문장에 대해서
    while len(item) < max_len:   # max_len보다 작으면
        item.append(0)

padded_np = np.array(encoded)
print(padded_np)
print(type(padded_np))
padded_np

[[ 1  5  0  0  0  0  0]
 [ 1  8  5  0  0  0  0]
 [ 1  3  5  0  0  0  0]
 [ 9  2  0  0  0  0  0]
 [ 2  4  3  2  0  0  0]
 [ 3  2  0  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  6  0  0  0  0]
 [ 1  4  2  0  0  0  0]
 [ 7  7  3  2 10  1 11]
 [ 1 12  3 13  0  0  0]]
<class 'numpy.ndarray'>


array([[ 1,  5,  0,  0,  0,  0,  0],
       [ 1,  8,  5,  0,  0,  0,  0],
       [ 1,  3,  5,  0,  0,  0,  0],
       [ 9,  2,  0,  0,  0,  0,  0],
       [ 2,  4,  3,  2,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  2,  0,  0,  0,  0],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0,  0,  0]])

- 길이가 7보다 짧은 문장 뒤에는 숫자 0이 붙어서 길이를 7로 만들어줌
- 컴퓨터는 이제 이 벡터들을 하나의 행렬로 보고 병렬처리 할 수 있음
- 0번 단어는 의미 없는 단어여서 자연어처리 과정에서 기계가 0번 단어를 무시함
- 이와 같이 데이터에 특정 값을 채워서 shape을 조정하는 행위를 <font color=red>패딩(Padding)</font>이라고 함.
- 특히, 숫자 0을 사용하여 패딩을 하는 것을 <ins>제로패딩(zero padding)</ins>이라고 함.

## 2) 케라스 전처리 도구로 패딩하기
- pad_sequences() : 케라스에서 제공하는 패딩을 위한 도구

In [16]:
# encoded를 패딩 이전의 값으로 돌려놓는 작업
encoded = tokenizer.texts_to_sequences(sentences)
print(encoded)

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


In [17]:
# 케라스의 pad_sequences를 이용한 패딩
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded = pad_sequences(encoded)
padded

array([[ 0,  0,  0,  0,  0,  1,  5],
       [ 0,  0,  0,  0,  1,  8,  5],
       [ 0,  0,  0,  0,  1,  3,  5],
       [ 0,  0,  0,  0,  0,  9,  2],
       [ 0,  0,  0,  2,  4,  3,  2],
       [ 0,  0,  0,  0,  0,  3,  2],
       [ 0,  0,  0,  0,  1,  4,  6],
       [ 0,  0,  0,  0,  1,  4,  6],
       [ 0,  0,  0,  0,  1,  4,  2],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 0,  0,  0,  1, 12,  3, 13]])

- 앞서 Numpy로 했던 패딩과 다른 이유 : 케라스의 pad_sequences는 앞쪽부터 0으로 채우는 패딩을 해서
- 만약 뒤로 0을 채워넣는 패딩을 하고 싶다면?
    - hyperparameter로 **padding='post'**를 쓸 것!

In [18]:
padded = pad_sequences(encoded, padding='post')
padded

array([[ 1,  5,  0,  0,  0,  0,  0],
       [ 1,  8,  5,  0,  0,  0,  0],
       [ 1,  3,  5,  0,  0,  0,  0],
       [ 9,  2,  0,  0,  0,  0,  0],
       [ 2,  4,  3,  2,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  2,  0,  0,  0,  0],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0,  0,  0]])

#### all(iterable) : 내장함수
    - iterable 의 모든 요소가 참이면 (또는 iterable 이 비어있으면) True 를 돌려줍니다.

In [23]:
(padded == padded_np).all()

True

- max_len의 인자로 정수를 주면, 해당 정수로 모든 문서의 길이를 동일하게 맞춤
- max_len을 최대 길이 문서보다 작은 길이로 했다면, **앞쪽이 잘림**

In [31]:
padded = pad_sequences(encoded, padding = 'post', maxlen = 5)
padded

array([[ 1,  5,  0,  0,  0],
       [ 1,  8,  5,  0,  0],
       [ 1,  3,  5,  0,  0],
       [ 9,  2,  0,  0,  0],
       [ 2,  4,  3,  2,  0],
       [ 3,  2,  0,  0,  0],
       [ 1,  4,  6,  0,  0],
       [ 1,  4,  6,  0,  0],
       [ 1,  4,  2,  0,  0],
       [ 3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0]])

### 0이 아닌 다른 숫자로 패딩하려면?

In [32]:
last_value = len(tokenizer.word_index) + 1 # 단어 집합의 크기보다 1 큰 숫자를 사용-->숫자 안겹치게 하려고
print(last_value)

padded = pad_sequences(encoded, padding = 'post', value = last_value)
padded

14


array([[ 1,  5, 14, 14, 14, 14, 14],
       [ 1,  8,  5, 14, 14, 14, 14],
       [ 1,  3,  5, 14, 14, 14, 14],
       [ 9,  2, 14, 14, 14, 14, 14],
       [ 2,  4,  3,  2, 14, 14, 14],
       [ 3,  2, 14, 14, 14, 14, 14],
       [ 1,  4,  6, 14, 14, 14, 14],
       [ 1,  4,  6, 14, 14, 14, 14],
       [ 1,  4,  2, 14, 14, 14, 14],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13, 14, 14, 14]])